# The following file consists of two programs: (1) calculating earnings surprise and (2) calculating the table means
# Before each file I give an introduction and then explain the process

#The first program generate earnings surprise in accordance to this WRDS SAS code
#https://github.com/jblocher/sas_util/blob/master/esurprises.sas
#Which is based on Livnat and Mendenhall (Journal of Accounting Research, Vol. 44, No.1, March 2006) (LM)
#The code calculates earnings surprise as the median IBES analyst forecast, in the last 90 days prior to the earnings, less the Actual earnings figure.
#It then merges it with Compustat date, and cleans it with accordance to LM for a more accurate sample
#In a different file I deflate each surprise by price at the last mutual fund last holding report date prior to the earnings announcement.
#I explain each step and what it does



Load python packages - you can skip this step

In [ ]:
import pandas as pd
import re
import numpy as np
import math
from datetime import datetime, timedelta
import os
import time
import glob
from scipy.stats import mstats

pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_colwidth', -1)
# Reduce decimal points to 2
pd.options.display.float_format = '{:,.4f}'.format

#set my desired working directory
os.chdir("/Users/ariel/Dropbox/PhD/Michael CSR/Analysis")

#Generate earnings surprise in accordance to https://github.com/jblocher/sas_util/blob/master/esurprises.sas
#Based on Livnat and Mendenhall (Journal of Accounting Research, Vol. 44, No.1, March 2006)

In [ ]:
#read ibes file
ibes=pd.read_csv('Data/CRSP/IBES_consensus_actual_Q_2011_2019.csv')
ibes=ibes.drop(columns=['TICKER','CNAME','CNAME','ANNTIMS_ACT']) #drop extra columns
ibes=ibes.dropna(subset=['ANNDATS_ACT','ACTUAL']) #drop missing actuals
ibes=ibes[ibes['FPI']==6] #keep quarterly reports - FPI = 6 the forecasts are made for the quarterly figures and are reported in the quarter before the publication of the quarterly report.


      Sample of the data and dictionary (I believe I have it all correctly):
      CUSIP - firm identifier
      ANALYS - analyst number
      VALUE - one quarter ahead forecast
      FPEDATS - end date of fiscal quarter the report is for   
      ANNDATS - Analyst forecast announcemnt data  
      ACTUAL - Actual earnings surprise
      ANNDATS_ACT - Earnings announcemnt date

In [ ]:
print(ibes.head())

In [ ]:
#keep only latest analyst report per firm
ibes=ibes.drop_duplicates(subset=['CUSIP','ANALYS','FPEDATS'],keep='last')

#change date format
ibes['FPEDATS']=pd.to_datetime(ibes['FPEDATS'],format='%Y%m%d')
ibes['ANNDATS_ACT']=pd.to_datetime(ibes['ANNDATS_ACT'],format='%Y%m%d')
ibes['ANNDATS']=pd.to_datetime(ibes['ANNDATS'],format='%Y%m%d')

#calculate date between forecast and acutal report
ibes['forecast_days']=(ibes['ANNDATS_ACT']-ibes['ANNDATS']).dt.days

#take only forecasts made within 90 days
ibes=ibes[(ibes['forecast_days']>0) & (ibes['forecast_days']<=90)]

#drop if number of analyst is forecasting is less than 3 per fiscal date
ibes=ibes.groupby(['CUSIP','FPEDATS']).filter(lambda x: len(x) > 3)

#take the median forecast, grouping by firm, fiscal date, announcment date, fiscal period 
ibes=ibes.groupby(['CUSIP','FPEDATS','ANNDATS_ACT','ACTUAL'])['VALUE'].median().reset_index()

#drop any duplicate firm-earnings_quarter_reports, keep the last one, which is the most updated - also possible to drop both, changes very little
ibes=ibes.drop_duplicates(subset=['CUSIP','FPEDATS'],keep='last')

#At this point we have analyst forecast and actual.
#Now we merge with Compustat to get share price and other cleaning factors according to LM
#we are left with fiscal quarter end date, Actual earning report date, the actual earnings figure, and the median forecast ('Value').
#(any problems with CUSIPS such as only zeros, or names will be dropped next when we merge it with Compustat based on the earnings report date).

In [ ]:
print(ibes.head())

In [ ]:
ibes['ANNDATS_ACT']=pd.to_datetime(ibes['ANNDATS_ACT'],format='%Y-%m-%d')#transform to date

compustat=pd.read_csv('Data/CRSP/CRSP_Compustat_Earning_Date_Querterly_2010_2020.csv') #load compustat
compustat['rdq']=pd.to_datetime(compustat['rdq'],format='%Y%m%d') #change date format
compustat['cusip'] = compustat['cusip'].str[:-1]#adjust CUSIP to 8 digits
compustat=compustat.dropna(subset=['rdq']) #drop missing earnings date

#merge  IBES with compustat
ibes_compustat=pd.merge(left=ibes,right=compustat,left_on=['ANNDATS_ACT','CUSIP'],right_on=['rdq','cusip'],how='left').dropna(subset=['cusip'])

#create market cap
ibes_compustat['mcap']=ibes_compustat['prccq']*ibes_compustat['cshoq'] #shares outstanding times share price

#filter from LM (2006):
ibes_compustat=ibes_compustat[(ibes_compustat.saleq>0)&(ibes_compustat.atq>0)&(ibes_compustat.fyr>0)&(ibes_compustat.prccq>1)&(ibes_compustat.mcap>5.0)].dropna(subset=['datafqtr'])

# we calculate the subsequent earnings surprise, which we calculate as the IBES actual EPS minus the most recent IBES consensus EPS scaled the latest fund report date share price (This is done a different file)
ibes_compustat['e_surprise_non_deflated']=(ibes_compustat['ACTUAL']-ibes_compustat['VALUE'])

#keep only relevant variables
ibes_compustat=ibes_compustat[['cusip','gvkey','FPEDATS','fyearq','fqtr','fyr','datafqtr','rdq','atq','saleq','prccq','mcap','e_surprise_non_deflated']]

#Finally we get the non-deflaed earnings surprise. 
#where rdq is earnings report date.
#As mentioned, I deflate by price at the last fund report date prior to the eanrings report  - which is the end of the month previous to rdq. This is done in a different file, 
#once I merge it with the holding report (The main reason is efficiency, because in order to have the price for the earnings report, I need to download the WHOLE CRSP daily file for a  about 8 years, which is extremely large. Once I merge this file with the holdings, it is then much more efficient to calculate this, since I have an exact list of stocks that meet the criteria)

In [ ]:
print(ibes_compustat.head())

# This is the second program . Here I calculate means for the tables based on Kyungran 2013.
#First, for each mutual fund-report date combination, we calculate the average equal-weighted earnings surprise for stocks in each trading category. This procedure results in one mutual fund average earnings surprise per trading category per report date. Meanings, I have for each mutual-fund an avergae per report date (mutual-fund-report-date avergae).
#Second, for each calendar quarter, we calculate the equal- weighted average of the mutual fund average earnings surprises by trading category for all report dates that fall in that quarter.
#Finally, to calculate yearly averages, we average across the four quarterly observations within each calendar year for each trading category
#trading categories are esg increase/decrease/first buys/least sales and non-esg increase/decrease/first buys/least sales
    

In [ ]:
def e_surprise_function(df):  
    df['e_surprise'] = mstats.winsorize(df['e_surprise'], limits=0.01) #winsorize earnings surprise per trade category
    df['e_report_mean'] = df.groupby(['crsp_portno','report_dt', 'esg_flag'])['e_surprise'].transform('mean') #take first mean
    df.drop_duplicates(subset=['crsp_portno','report_dt', 'esg_flag'],inplace=True)
    df['e_q_mean'] = df.groupby(['quarter_of_report_date', 'esg_flag'])['e_report_mean'].transform('mean') #take second mean
    df.drop_duplicates(subset=['quarter_of_report_date', 'esg_flag'],inplace=True)
    df['e_y_mean']=df.groupby(['year_of_report_date', 'esg_flag'])['e_q_mean'].transform('mean') #Take last mean
    